In [39]:
# Load environment variables
# python -m ipykernel install --user --name=webnearme-venv
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())



True

In [40]:
# Run basic query with OpenAI wrapper

# from langchain.llms import OpenAI
# llm = OpenAI(model_name="text-davinci-003")
# llm("explain large language models in one sentence")

In [41]:
# import schema for chat messages and ChatOpenAI in order to query chatmodels GPT-3.5-turbo or GPT-4

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI

In [42]:
# chat = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0.3)
# messages = [
#     SystemMessage(content="You are an expert data scientist"),
#     HumanMessage(content="Write a Python script that trains a neural network on simulated data ")
# ]
# response=chat(messages)

# print(response.content,end='\n')

In [43]:
# # Import prompt and define PromptTemplate

# from langchain import PromptTemplate

# template = """
# You are an expert data scientist with an expertise in building deep learning models. 
# Explain the concept of {concept} in a couple of lines
# """

# prompt = PromptTemplate(
#     input_variables=["concept"],
#     template=template,
# )

In [44]:
# Run LLM with PromptTemplate

# llm(prompt.format(concept="vector")

In [45]:
# Import LLMChain and define chain with language model and prompt as arguments.

# from langchain.chains import LLMChain
# chain = LLMChain(llm=llm, prompt=prompt)

# # Run the chain only specifying the input variable.
# print(chain.run("autoencoder"))

In [46]:
# # Define a second prompt 

# second_prompt = PromptTemplate(
#     input_variables=["ml_concept"],
#     template="Turn the concept description of {ml_concept} and explain it to me like I'm five in 500 words",
# )
# chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [47]:
# # Define a sequential chain using the two chains above: the second chain takes the output of the first chain as input

# from langchain.chains import SimpleSequentialChain
# overall_chain = SimpleSequentialChain(chains=[chain, chain_two], verbose=True)

# # Run the chain specifying only the input variable for the first chain.
# explanation = overall_chain.run("autoencoder")
# print(explanation)

In [48]:
# Import utility for splitting up texts and split up the explanation given above into document chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap  = 0,
)

texts = text_splitter.create_documents([explanation])
len(texts)

14

In [49]:
# Import and initialize Pinecone client

import os
import pinecone
from langchain.vectorstores import Pinecone


pinecone.init(
    api_key=os.getenv('PINECONE_API_KEY'),  
    environment=os.getenv('PINECONE_ENV')  
)

In [50]:
import requests
from requests.packages.urllib3.util.ssl_ import create_urllib3_context

CIPHERS = (
    'ECDHE+AESGCM:ECDHE+CHACHA20:DHE+AESGCM:DHE+CHACHA20:ECDH+AESGCM:ECDH+CHACHA20:DH+AESGCM:DH+CHACHA20:'
    'ECDHE+AES:!aNULL:!eNULL:!EXPORT:!DES:!MD5:!PSK:!RC4:!HMAC_SHA1:!SHA1:!DHE+AES:!ECDH+AES:!DH+AES'
)

requests.packages.urllib3.util.ssl_.DEFAULT_CIPHERS = CIPHERS
# Skip the following two lines if they cause errors
# requests.packages.urllib3.contrib.pyopenssl.DEFAULT_SSL_CIPHER_LIST = CIPHERS
# requests.packages.urllib3.contrib.pyopenssl.inject_into_urllib3()
requests.packages.urllib3.util.ssl_.create_default_context = create_urllib3_context


In [51]:
# # Initialize Pinecone
# index_name = "websitenearme-fast-api"
# DIMENSIONS=128

# # Create and configure index if doesn't already exist
# if index_name not in pinecone.list_indexes():
#     pinecone.create_index(
#         name=index_name, 
#         metric="cosine",
#         dimension=DIMENSIONS)
#     docsearch = Pinecone.from_documents(texts[0], embeddings, index_name=index_name)

# else:
#     docsearch = Pinecone.from_existing_index(index_name, embeddings)

# print(docsearch)

[]

## we created the index up to this point
## next we will insert the data into pinecone

# we will create a hybrid index that can search semantic and keyword

In [54]:
index = pinecone.Index('websitenearme-fast-api')

In [55]:
all_sentences = ["Fast websites, fast!","HIRE US","MAIN PRODUCT/SERVICE 1","OUR EX1: WEBSITE DESIGN","","Initial website, same template as this site	1 design + hosting and domain (godaddy or transferred to godaddy)= ~$550","AI-powered SEO initial setup	$0, included","multilingual	$0, included","chatbot + maintenance	NOT INCLUDED","Website design","MAIN PRODUCT/SERVICE 2","OUR EX2: CHATBOTS FOR OUR WEBSITES","","Website + 1 site-specific chatbot	$100 * 1 time, chatbot setup, $20/month chatGPT subscription","Train model with AI-generated + your questions & answers	$100/ training round","Multiple chatbots with different personalities/ purposes	$100 each","Connect chatbot to your existing website (i.e., we did not build your website)	Please contact us with specifics, these start at $500/each and we reserve the right to say no","Chatbot types and prices","MAIN PRODUCT/SERVICE 3","EX3: SUBSCRIPTIONS","","monthly site health + maintenance + minor updates + 3 emergency updates	$50/month","monthly site health + maintenance + minor updates + 1 emergency update/week	$250/month","emergency update	$50","content generation- 4 posts, 1 per week	$100/month","Subscriptions and prices"]

In [59]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('flax-sentence-embeddings/all_datasets_v3_mpnet-base')

all_embeddings = model.encode(all_sentences)
print(all_embeddings.shape)
print(all_embeddings.shape[1])
print(all_embeddings[0:1])

(26, 768)
768
[[-2.35444633e-03  5.48605137e-02 -3.57232727e-02 -8.66033509e-03
   1.22838309e-02 -2.31732037e-02  2.15398464e-02  3.68994251e-02
  -1.17613141e-04 -4.66104783e-02 -5.90305449e-03 -6.81372806e-02
   3.35595235e-02  5.76601364e-02  5.46999602e-03  1.67380031e-02
   4.78123734e-03  1.83588676e-02 -1.51509894e-02  1.55020389e-03
   1.04834959e-02 -4.37937155e-02 -2.71082576e-02 -1.38526736e-02
   2.51252055e-02 -3.59955840e-02 -7.81918876e-03  3.21647860e-02
  -4.07713130e-02 -5.99804297e-02  2.53628823e-03 -1.73783060e-02
  -2.66849659e-02  2.87912805e-02  3.07434567e-09  1.53542096e-02
  -1.65473588e-03 -7.35954242e-03 -9.00205299e-02  1.03918694e-01
  -3.22410986e-02  9.26716160e-03 -3.19100432e-02  1.39510101e-02
   1.24675678e-02 -4.18858081e-02  5.70722297e-03 -5.45279644e-02
  -4.88943644e-02  5.97022660e-02 -3.53687368e-02  1.49220414e-02
  -3.00463904e-02 -2.08735559e-02  3.38984355e-02 -3.03387512e-02
  -1.00002084e-02 -5.85185178e-02  6.18790351e-02  6.68968447e

# the above is sufficient for upserting semantic searches, however the next step allows us to be able to do keyword searches

In [57]:
%pip install sacremoses
from transformers import AutoTokenizer

# transfo-xl tokenizer uses word-level encodings
tokenizer = AutoTokenizer.from_pretrained('transfo-xl-wt103')

all_tokens = [tokenizer.tokenize(sentence.lower()) for sentence in all_sentences]
all_tokens[0]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


['fast', 'websites', ',', 'fast', '!']

## we will connect to pinecone

In [60]:
import pinecone

# connect to pinecone environment
pinecone.init(
    api_key=os.getenv('PINECONE_API_KEY'),  
    environment=os.getenv('PINECONE_ENV') 
)

pinecone.list_indexes()  # check if keyword-search index already exists


# # Initialize Pinecone
INDEX_NAME = "websitenearme-fast-api"
DIMENSIONS=all_embeddings.shape[1]

# # Create and configure index if doesn't already exist
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=INDEX_NAME, 
        metric="eucladian",
        dimension=DIMENSIONS
        )

index = pinecone.Index(INDEX_NAME)
pinecone.list_indexes()
pinecone.describe_index(index_name)

IndexDescription(name='websitenearme-fast-api', metric='cosine', replicas=1, dimension=768.0, shards=1, pods=1, pod_type='p1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

## now we will upsert the data

In [71]:
# only do this if i want a local json file
import json
from datetime import datetime

# Assume all_embeddings and all_tokens are already defined

# reformat the data
upserts = {'vectors': []}
for i, (embedding, tokens) in enumerate(zip(all_embeddings, all_tokens)):
    vector = {
        'id': f'{i}',
        'metadata': {
            'tokens': tokens,
            'time_stamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')  # Place timestamp within metadata
        },
        'values': embedding.tolist()
    }
    upserts['vectors'].append(vector)

# Add namespace at the root level
upserts['namespace'] = 'websitenearme'

# save to JSON
with open('./upsert.json', 'w') as f:
    json.dump(upserts, f, indent=4)



In [74]:
# upsert data straight into pinecone
import json
from datetime import datetime
from tqdm.auto import tqdm  # for progress bar

# Assume all_embeddings and all_tokens are already defined

# reformat the data
# data means the zipped embeddings and all tokens
# this correctly formats the json dump
# and is working to upload straight 
# into pincone as of Aguust 9, 2023
upserts = {'vectors': []}
for i, (embedding, tokens) in enumerate(zip(all_embeddings, all_tokens)):
    vector = {
        'id': f'{i}',
        'metadata': {
            'tokens': tokens,
            'time_stamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')  # Place timestamp within metadata
        },
        'values': embedding.tolist()
    }
    upserts['vectors'].append(vector)

# Add namespace at the root level
upserts['namespace'] = 'websitenearme'

# Use upserts['vectors'] as the dataset
dataset = upserts['vectors']

# Insert data as batches
batch_size = 100
for i in tqdm(range(0, len(dataset), batch_size)):
    # set end of current batch
    i_end = i + batch_size
    if i_end > len(dataset):
        # correct if batch is beyond dataset size
        i_end = len(dataset)
    batch = dataset[i: i_end]
    # Upsert the batch (assuming the structure of batch matches the expected format)
    index.upsert(vectors=batch)


  0%|          | 0/1 [00:00<?, ?it/s]

In [67]:
query_sentence = "prices"
xq = model.encode(query_sentence).tolist()

In [68]:
result = index.query(xq, top_k=1, includeMetadata=True)
result

{'matches': [{'id': '25',
              'metadata': {'time_stamp': datetime.datetime(2023, 8, 9, 10, 26, 7),
                           'tokens': ['subscriptions', 'and', 'prices']},
              'score': 0.488167524,
              'values': []}],
 'namespace': ''}

In [70]:
# use this to check what's in pinecone

In [ ]:
# create embeddings
import openai

openai.api_key = os.getenv('OPENAI_API_KEY')

MODEL = "text-embedding-ada-002"

res = openai.Embedding.create(
    input=input_lst # must be a list
    , engine=MODEL
)

# extract embeddings to a list
embeds = [record['embedding'] for record in res['data']]


InvalidRequestError: '$.input' is invalid. Please check the API reference: https://platform.openai.com/docs/api-reference.

In [ ]:
print((embeds))

[[-0.006369457580149174, -0.011372314766049385, 0.0011751734418794513, -0.012635905295610428, 0.006651018280535936, -0.005380560643970966, -0.008288878947496414, -0.021329956129193306, 9.689386934041977e-05, -0.01825338788330555, -0.00976535677909851, 0.026604073122143745, -0.008302613161504269, -0.0032791546545922756, 0.007224441505968571, 0.009669214487075806, 0.01995648816227913, -0.007008119951933622, 0.006990951951593161, -0.015657534822821617, -0.035270657390356064, 0.01731942966580391, -0.006153136491775513, 0.004669791087508202, -0.03705616295337677, -0.012526027858257294, -0.0031400909647345543, -0.007279380224645138, -0.008419358171522617, 0.01251916028559208, -0.003406200557947159, -0.005765131674706936, -0.017566654831171036, 0.004233714658766985, -0.002431038301438093, 0.023554975166916847, -0.002642208943143487, 0.014091781340539455, 0.03123266063630581, 0.004288653377443552, 0.026755155995488167, 0.02027238719165325, 0.008055388927459717, -0.00388004700653255, -0.0386768

In [ ]:
index.upsert()

In [ ]:
# Do a simple vector similarity search

query = "What is magical about an autoencoder?"
result = docsearch.similarity_search(query)

# print(result)

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'application/json', 'date': 'Tue, 08 Aug 2023 23:51:35 GMT', 'x-envoy-upstream-service-time': '0', 'content-length': '109', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Query vector dimension 1024 does not match the dimension of the index 128","details":[]}


In [ ]:
# Import Python REPL tool and instantiate Python agent

from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.llms.openai import OpenAI

agent_executor = create_python_agent(
    llm=OpenAI(temperature=0, max_tokens=1000),
    tool=PythonREPLTool(),
    verbose=True
)

In [ ]:
# Execute the Python agent

agent_executor.run("Find the roots (zeros) if the quadratic function 3 * x**2 + 2*x -1")